In [2]:
import streamlit as st
import gspread as gs 

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss_portfolio = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key).worksheet(st.secrets['portfolio'].worksheet_name)

In [66]:
import yfinance as yf
import pandas as pd

df_portfolio = pd.DataFrame(ss_portfolio.get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'})
ticker = {
    'Deposit€' : '€',
    'Deposit$' : 'EURUSD=X',
    'DepositUSDT' : 'EURUSD=X',
    'BTC' : 'BTC-EUR',
    'ESE' : 'ESE.PA',
    'GOLD' : 'GOLD.PA',
    'GREENBULL' : '€',
}
market: pd.DataFrame = yf.download('SPY EURUSD=X BTC-EUR ESE.PA GOLD.PA', start=min(df_portfolio['Date']))['Close'].reset_index().ffill()
market['€'] = 1
df_portfolio['Total€'] = [df_portfolio.loc[idx]['Total'] / (1 if df_portfolio.loc[idx]['Currency'] == "€" else market[market['Date'] == df_portfolio.loc[idx]['Date']]['EURUSD=X'].values[0]) for idx in df_portfolio.index]

market

[*********************100%***********************]  5 of 5 completed
dict_values(['€', 'EURUSD=X', 'EURUSD=X', 'BTC-EUR', 'ESE.PA', 'GOLD.PA', '€'])


,Date,BTC-EUR,ESE.PA,EURUSD=X,GOLD.PA,SPY,€
0,2021-04-30,48047.050781,16.617201,1.212709,58.556999,417.299988,1
1,2021-05-01,48110.046875,16.617201,1.212709,58.556999,417.299988,1
2,2021-05-02,47088.570312,16.617201,1.212709,58.556999,417.299988,1
3,2021-05-03,47432.769531,16.637800,1.202776,59.213001,418.200012,1
4,2021-05-04,44387.261719,16.475599,1.206127,59.227001,415.619995,1
...,...,...,...,...,...,...,...
576,2022-11-27,15862.076172,18.905701,1.040810,66.992996,402.329987,1
577,2022-11-28,15678.680664,18.754200,1.036753,66.849998,395.910004,1
578,2022-11-29,15928.907227,18.610600,1.034554,67.339996,395.230011,1
579,2022-11-30,16471.539062,18.721399,1.032450,67.660004,407.679993,1


In [73]:
import numpy as np

# Liste des couple Portfolio / Asset 
position = []

# Dataframe result
df = df_portfolio.astype({'Date': 'datetime64[ns]'})

for portfolio in np.unique(df['Portfolio']):
    # Dataframe of Portfolio
    dfp: pd.DataFrame = df[df['Portfolio'] == portfolio]

    for asset in np.unique(dfp['Asset']):
        # Dataframe of Asset
        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Total€': "sum", 'Amount': "sum"})

        amt, tot, pru = pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in dfa.index:
            amt[idx] = np.sum(dfa.loc[:idx]['Amount'])
            tot[idx] = np.sum(dfa.loc[:idx]['Total€'])
            pru[idx] = tot[idx] / amt[idx]

        col = f'{portfolio}_{asset}'
        dfp = dfp.merge(pd.DataFrame({
            'Date': pd.to_datetime(dfa.index),
            f'{col}_Amount': amt,
            f'{col}_Total': tot,
            f'{col}_PRU': pru,
        }), on='Date', how='left')

        position.append(col)

    cols_inv, cols_dep = [], []
    for col in dfp.columns:
        if portfolio in col and 'Total' in col:
            if 'Deposit' in col:
                cols_dep.append(col)
            else:
                cols_inv.append(col)
    
    dfp = dfp.ffill()
    dfp[f'{portfolio}_Invested'] = dfp[cols_inv].sum(axis=1)
    dfp[f'{portfolio}_Deposit'] = dfp[cols_dep].sum(axis=1)

    df = df.merge(dfp[
        ['Date'] + [col for col in dfp.columns if portfolio in col]
    ], on='Date', how='left')

df = pd.merge(market, df, on='Date', how='left').ffill().fillna(0)

cols_val, cols_pnl = [], []
for portfolio in np.unique(df['Portfolio']):
    for asset in np.unique(df[df['Portfolio'] == portfolio]['Asset']):
        col = f'{portfolio}_{asset}'
        df[f'{col}_Value'] = df[f'{col}_Amount'] * df[f'{ticker[asset]}']
        df[f'{col}_PnL'] = df[f'{col}_Value'] - df[f'{col}_Total']
        cols_val.append(f'{col}_Value')
        cols_pnl.append(f'{col}_PnL')

    df[f'{portfolio}_Value'] = df[cols_val].sum(axis=1)
    df[f'{portfolio}_PnL'] = df[cols_pnl].sum(axis=1)


print('\n', df.shape, df.columns.to_list())
df[col ]


 (650, 66) ['Date', 'BTC-EUR', 'ESE.PA', 'EURUSD=X', 'GOLD.PA', 'SPY', '€', 'Portfolio', 'Account', 'Asset', 'Price', 'Amount', 'Fees', 'Total', 'Currency', 'Total€', 'Out', 'Profit', 'DMA_Deposit€_Amount', 'DMA_Deposit€_Total', 'DMA_Deposit€_PRU', 'DMA_Invested', 'DMA_Deposit', 'ZEN_BTC_Amount', 'ZEN_BTC_Total', 'ZEN_BTC_PRU', 'ZEN_Deposit$_Amount', 'ZEN_Deposit$_Total', 'ZEN_Deposit$_PRU', 'ZEN_DepositUSDT_Amount', 'ZEN_DepositUSDT_Total', 'ZEN_DepositUSDT_PRU', 'ZEN_Deposit€_Amount', 'ZEN_Deposit€_Total', 'ZEN_Deposit€_PRU', 'ZEN_ESE_Amount', 'ZEN_ESE_Total', 'ZEN_ESE_PRU', 'ZEN_GOLD_Amount', 'ZEN_GOLD_Total', 'ZEN_GOLD_PRU', 'ZEN_GREENBULL_Amount', 'ZEN_GREENBULL_Total', 'ZEN_GREENBULL_PRU', 'ZEN_Invested', 'ZEN_Deposit', 'DMA_Deposit€_Value', 'DMA_Deposit€_PnL', 'DMA_Value', 'DMA_PnL', 'ZEN_BTC_Value', 'ZEN_BTC_PnL', 'ZEN_Deposit$_Value', 'ZEN_Deposit$_PnL', 'ZEN_DepositUSDT_Value', 'ZEN_DepositUSDT_PnL', 'ZEN_Deposit€_Value', 'ZEN_Deposit€_PnL', 'ZEN_ESE_Value', 'ZEN_ESE_PnL', '

,Date,BTC-EUR,ESE.PA,EURUSD=X,GOLD.PA,SPY,€,Portfolio,Account,Asset,...,ZEN_Deposit€_Value,ZEN_Deposit€_PnL,ZEN_ESE_Value,ZEN_ESE_PnL,ZEN_GOLD_Value,ZEN_GOLD_PnL,ZEN_GREENBULL_Value,ZEN_GREENBULL_PnL,ZEN_Value,ZEN_PnL
0,2021-04-30,48047.050781,16.617201,1.212709,58.556999,417.299988,1,ZEN,eToro,Deposit$,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,598.0,104.889198,1323.200095,330.089293
1,2021-04-30,48047.050781,16.617201,1.212709,58.556999,417.299988,1,ZEN,eToro,Deposit$,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,598.0,104.889198,1323.200095,330.089293
2,2021-04-30,48047.050781,16.617201,1.212709,58.556999,417.299988,1,ZEN,eToro,GREENBULL,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,598.0,104.889198,1323.200095,330.089293
3,2021-04-30,48047.050781,16.617201,1.212709,58.556999,417.299988,1,ZEN,eToro,GREENBULL,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,598.0,104.889198,1323.200095,330.089293
4,2021-05-01,48110.046875,16.617201,1.212709,58.556999,417.299988,1,ZEN,eToro,GREENBULL,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,598.0,104.889198,1323.200095,330.089293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2022-11-27,15862.076172,18.905701,1.040810,66.992996,402.329987,1,ZEN,Binance,BTC,...,5500.0,0.0,1039.813538,53.413538,1741.817902,-73.372098,5028.0,568.188404,30293.134479,1097.227626
646,2022-11-28,15678.680664,18.754200,1.036753,66.849998,395.910004,1,ZEN,Binance,BTC,...,5500.0,0.0,1031.480999,45.080999,1738.099960,-77.090040,5028.0,568.188404,30241.297641,1045.390788
647,2022-11-29,15928.907227,18.610600,1.034554,67.339996,395.230011,1,ZEN,Binance,BTC,...,5500.0,0.0,1023.582973,37.182973,1750.839905,-64.350095,5028.0,568.188404,30243.891805,1047.984952
648,2022-11-30,16471.539062,18.721399,1.032450,67.660004,407.679993,1,ZEN,Binance,BTC,...,5500.0,0.0,1029.676962,43.276962,1759.160095,-56.029905,5028.0,568.188404,30272.901638,1076.994785


In [72]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['ZEN_Deposit']))
fig.add_trace(go.Scatter(x=df['Date'], y=df['ZEN_Invested']))
fig.add_trace(go.Scatter(x=df['Date'], y=df['ZEN_Value']))
fig.show()

In [29]:
tic = yf.download('SPY EURUSD=X BTC-USD ESE.PA GOLD.PA', start=min(df_portfolio['Date']))['Close'].ffill()
tic

[*********************100%***********************]  5 of 5 completed


,BTC-USD,ESE.PA,EURUSD=X,GOLD.PA,SPY
Date,,,,,
2021-04-30,57750.175781,16.617201,1.212709,58.556999,417.299988
2021-05-01,57828.050781,16.617201,1.212709,58.556999,417.299988
2021-05-02,56631.078125,16.617201,1.212709,58.556999,417.299988
2021-05-03,57200.292969,16.637800,1.202776,59.213001,418.200012
2021-05-04,53333.539062,16.475599,1.206127,59.227001,415.619995
...,...,...,...,...,...
2022-11-26,16464.281250,18.905701,1.040810,66.992996,402.329987
2022-11-27,16444.626953,18.905701,1.040810,66.992996,402.329987
2022-11-28,16217.322266,18.754200,1.036753,66.849998,395.910004


In [31]:
import plotly.express as px

fig = px.line(tic[['ESE.PA', 'GOLD.PA']])
fig.show()